# 🎯 Construção da Pré-lista UNIVERSE_CANDIDATES

Este notebook executa o fluxo completo:
1. **Ingestão** de dados de mercado (Yahoo Finance)
2. **Pipeline** UNIVERSE_CANDIDATES (filtros e classificação)
3. **Resumo** da pré-lista gerada

---


## 1. Setup e Imports


In [10]:
# Instala dependências necessárias (execute apenas uma vez)
# Descomente a linha abaixo se pyarrow não estiver instalado
# !pip install pyarrow yfinance polars pyyaml --quiet

# Verifica se pyarrow está instalado
try:
    import pyarrow
    print(f"✅ pyarrow versão: {pyarrow.__version__}")
except ImportError:
    print("❌ pyarrow NÃO está instalado!")
    print("   Execute: pip install pyarrow")
    print("   Ou descomente a linha !pip install acima e re-execute esta célula")


✅ pyarrow versão: 22.0.0


In [11]:
import sys
import os
import logging
from pathlib import Path
from datetime import datetime
import json

# Configura logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Define o diretório raiz do projeto
# Detecta automaticamente se estamos em notebooks/ ou na raiz
current_dir = Path.cwd()
if current_dir.name == 'notebooks':
    project_root = current_dir.parent
elif (current_dir / 'pyproject.toml').exists():
    project_root = current_dir
else:
    # Fallback: procura pyproject.toml subindo níveis
    project_root = current_dir
    for _ in range(5):
        if (project_root / 'pyproject.toml').exists():
            break
        project_root = project_root.parent

# Muda para o diretório do projeto (importante para paths relativos)
os.chdir(project_root)

# Adiciona módulos ao path
sys.path.insert(0, str(project_root / 'modules'))

print(f"📁 Project root: {project_root}")
print(f"📂 Working directory: {Path.cwd()}")
print(f"📅 Data/hora: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


📁 Project root: /home/wilson/PortfolioZero
📂 Working directory: /home/wilson/PortfolioZero
📅 Data/hora: 2025-12-02 17:07:50


In [12]:
# Imports dos módulos do PortfolioZero
from portfoliozero.core.data.market_data_ingestion import (
    fetch_and_store_universe_market_data,
    validate_raw_market_data,
    load_data_source_config,
)

from portfoliozero.core.data.universe_candidates_pipeline import (
    build_universe_candidates,
    load_universe_candidates,
    validate_universe_candidates,
    get_pipeline_metadata,
)

import polars as pl

print("✅ Imports carregados com sucesso!")


✅ Imports carregados com sucesso!


## 2. Configuração

Carrega a configuração para ver os tickers que serão baixados.


In [13]:
# Caminho do arquivo de configuração
config_path = str(project_root / 'config' / 'experiments' / 'universe_data_sources_v1.yaml')
print(f"📄 Config path: {config_path}")

# Carrega configuração
config = load_data_source_config(config_path)

print(f"\n📊 Provider: {config.get('provider', 'N/A')}")
print(f"📈 Tickers configurados: {len(config.get('universe', []))}")
print(f"📅 Período: {config.get('date_range', {}).get('start', 'N/A')} a {config.get('date_range', {}).get('end', 'N/A')}")
print(f"\n🏷️ Primeiros 10 tickers:")
for ticker in config.get('universe', [])[:10]:
    print(f"   - {ticker}")
if len(config.get('universe', [])) > 10:
    print(f"   ... e mais {len(config.get('universe', [])) - 10} tickers")


INFO - Configuração carregada: /home/wilson/PortfolioZero/config/experiments/universe_data_sources_v1.yaml
INFO -   Provider: yahoo_finance
INFO -   Tickers: 76


📄 Config path: /home/wilson/PortfolioZero/config/experiments/universe_data_sources_v1.yaml

📊 Provider: yahoo_finance
📈 Tickers configurados: 76
📅 Período: 2022-01-01 a today

🏷️ Primeiros 10 tickers:
   - ITUB4.SA
   - BBDC4.SA
   - BBAS3.SA
   - SANB11.SA
   - ITSA4.SA
   - B3SA3.SA
   - BBSE3.SA
   - VALE3.SA
   - PETR4.SA
   - PETR3.SA
   ... e mais 66 tickers


## 3. Ingestão de Dados de Mercado 📥

Baixa dados OHLCV do Yahoo Finance para todos os tickers configurados.

⚠️ **Atenção:** Esta etapa pode demorar 5-10 minutos dependendo da quantidade de tickers e conexão.


In [14]:
# Define se deve sobrescrever dados existentes
from pickle import TRUE


OVERWRITE_EXISTING = False  # Mude para True se quiser re-baixar tudo

print("="*60)
print("INGESTÃO DE DADOS DE MERCADO")
print("="*60)
print(f"\n⚙️ Sobrescrever existentes: {OVERWRITE_EXISTING}")
print(f"🚀 Iniciando ingestão...\n")
# Executa ingestão (usa o config_path definido na célula anterior)
files = fetch_and_store_universe_market_data(config_path=config_path, overwrite=OVERWRITE_EXISTING)

print(f"\n✅ Ingestão concluída!")
print(f"📁 Arquivos gerados/atualizados: {len(files)}")


INFO - Configuração carregada: /home/wilson/PortfolioZero/config/experiments/universe_data_sources_v1.yaml
INFO -   Provider: yahoo_finance
INFO -   Tickers: 76
INFO - ============================================================
INFO - Iniciando ingestão de dados de mercado
INFO -   Provider: yahoo_finance
INFO -   Tickers: 76
INFO -   Período: 2022-01-01 a 2025-12-02
INFO - ============================================================
INFO - [1/76] Processando ITUB4.SA...
INFO -   Arquivo já existe, pulando: /home/wilson/PortfolioZero/data/raw/market/prices/ITUB4_SA.parquet
INFO - [2/76] Processando BBDC4.SA...
INFO -   Arquivo já existe, pulando: /home/wilson/PortfolioZero/data/raw/market/prices/BBDC4_SA.parquet
INFO - [3/76] Processando BBAS3.SA...
INFO -   Arquivo já existe, pulando: /home/wilson/PortfolioZero/data/raw/market/prices/BBAS3_SA.parquet
INFO - [4/76] Processando SANB11.SA...
INFO -   Arquivo já existe, pulando: /home/wilson/PortfolioZero/data/raw/market/prices/SANB11_SA

INGESTÃO DE DADOS DE MERCADO

⚙️ Sobrescrever existentes: False
🚀 Iniciando ingestão...


✅ Ingestão concluída!
📁 Arquivos gerados/atualizados: 65


In [15]:
# Valida dados brutos
validation = validate_raw_market_data()

print("\n📋 Validação dos dados brutos:")
print(f"   Válido: {'✅' if validation['valid'] else '❌'}")
print(f"   Arquivos: {validation.get('files', 0)}")
print(f"   Registros totais: {validation.get('total_records', 0):,}")
print(f"   Tickers únicos: {len(validation.get('tickers', []))}")

if validation.get('errors'):
    print(f"\n⚠️ Erros encontrados:")
    for err in validation['errors'][:5]:
        print(f"   - {err}")



📋 Validação dos dados brutos:
   Válido: ✅
   Arquivos: 65
   Registros totais: 63,626
   Tickers únicos: 65


## 4. Execução do Pipeline UNIVERSE_CANDIDATES 🔧

Aplica os filtros de seleção e gera a pré-lista de 60-80 candidatos.


In [16]:
print("="*60)
print("PIPELINE UNIVERSE_CANDIDATES")
print("="*60)

# Executa pipeline
output_path = build_universe_candidates(force_refresh=True)

print(f"\n✅ Pipeline concluído!")
print(f"📁 Arquivo gerado: {output_path}")


INFO - ============================================================
INFO - Iniciando pipeline UNIVERSE_CANDIDATES
INFO - Project root: /home/wilson/PortfolioZero
INFO - ============================================================
INFO - Configurações carregadas com sucesso
INFO - ----------------------------------------
INFO - ETAPA 1: Leitura de dados brutos
INFO - Encontrados 65 arquivos Parquet em /home/wilson/PortfolioZero/data/raw/market
INFO - Dados brutos carregados: 63626 registros, 10 colunas
INFO - ----------------------------------------
INFO - ETAPA 2: Normalização de identificadores
INFO - Dados normalizados: 63626 registros
INFO - ----------------------------------------
INFO - ETAPA 3: Cálculo de métricas por ticker
INFO - Métricas calculadas para 65 tickers únicos
INFO - ----------------------------------------
INFO - ETAPA 4: Aplicação de filtros da pré-lista
INFO - Após filtro histórico mínimo: 65 ativos (removidos: 0)
INFO - Após filtro trading ratio: 65 ativos (remo

PIPELINE UNIVERSE_CANDIDATES

✅ Pipeline concluído!
📁 Arquivo gerado: /home/wilson/PortfolioZero/data/universe/UNIVERSE_CANDIDATES.parquet


## 5. Carrega e Analisa Resultado 📊


In [17]:
# Carrega resultado
df = load_universe_candidates(output_path)

# Valida
result = validate_universe_candidates(df)

print(f"📊 Candidatos carregados: {len(df)}")
print(f"✅ Validação: {'OK' if result.is_valid else 'FALHOU'}")
if result.warnings:
    print(f"⚠️ Warnings: {len(result.warnings)}")
    for w in result.warnings:
        print(f"   - {w}")


INFO - Carregado UNIVERSE_CANDIDATES: 31 registros de /home/wilson/PortfolioZero/data/universe/UNIVERSE_CANDIDATES.parquet


📊 Candidatos carregados: 31
✅ Validação: OK
⚠️ Warnings: 1
   - Universo pequeno: 31 ativos (esperado 60-80)


In [18]:
# Mostra primeiros registros
print("\n📋 Primeiros 15 candidatos (ordenados por volume):")
df.head(15)



📋 Primeiros 15 candidatos (ordenados por volume):


ticker,tipo_instrumento,setor,date_first,date_last,history_days,trading_days_ratio_252d,avg_volume_21d_brl,avg_price_recent_brl,last_price,annualized_vol_60d,volatility_class,liquidity_class
str,str,str,date,date,u32,f64,f64,f64,f64,f64,str,str
"""CSAN3.SA""","""ACAO_ON""","""Energia""",2022-01-03,2025-12-01,980,1.0,4.6898e7,6.092381,6.15,0.567496,"""ALTA""","""ALTA"""
"""PETR4.SA""","""ACAO_PN""","""Commodities""",2022-01-03,2025-12-01,980,1.0,4.3878e7,31.980477,31.85,0.179207,"""BAIXA""","""ALTA"""
"""B3SA3.SA""","""ACAO_ON""","""Financeiro""",2022-01-03,2025-12-01,980,1.0,3.7703e7,13.842381,14.89,0.25815,"""MEDIA""","""ALTA"""
"""ABEV3.SA""","""ACAO_ON""","""Consumo""",2022-01-03,2025-12-01,980,1.0,3.0933e7,13.484286,14.01,0.195297,"""BAIXA""","""ALTA"""
"""ITSA4.SA""","""ACAO_PN""","""Financeiro""",2022-01-03,2025-12-01,980,1.0,3.0450e7,11.90381,12.24,0.161098,"""BAIXA""","""ALTA"""
…,…,…,…,…,…,…,…,…,…,…,…,…
"""LREN3.SA""","""ACAO_ON""","""Consumo""",2022-01-03,2025-12-01,980,1.0,1.8988e7,15.066667,15.76,0.336896,"""MEDIA""","""ALTA"""
"""MGLU3.SA""","""ACAO_ON""","""Consumo""",2022-01-03,2025-12-01,980,1.0,1.7150e7,9.235238,10.34,0.578644,"""ALTA""","""ALTA"""
"""RAIL3.SA""","""ACAO_ON""","""Utilidades""",2022-01-03,2025-12-01,980,1.0,1.7062e7,15.958095,16.58,0.354262,"""MEDIA""","""ALTA"""


## 6. Resumo da Pré-lista 📈


In [19]:
# Constantes
TARGET_MIN = 60
TARGET_MAX = 80

total = len(df)
in_range = TARGET_MIN <= total <= TARGET_MAX

print("="*60)
print("RESUMO DA PRÉ-LISTA UNIVERSE_CANDIDATES")
print("="*60)

print(f"\n📊 Total de candidatos: {total}")
print(f"🎯 Intervalo alvo: {TARGET_MIN} - {TARGET_MAX}")

if in_range:
    print(f"✅ Dentro do intervalo: SIM")
else:
    print(f"❌ Dentro do intervalo: NÃO")
    if total < TARGET_MIN:
        print(f"   ⚠️ Faltam {TARGET_MIN - total} candidatos para atingir o mínimo")
    else:
        print(f"   ⚠️ Excedem {total - TARGET_MAX} candidatos acima do máximo")


RESUMO DA PRÉ-LISTA UNIVERSE_CANDIDATES

📊 Total de candidatos: 31
🎯 Intervalo alvo: 60 - 80
❌ Dentro do intervalo: NÃO
   ⚠️ Faltam 29 candidatos para atingir o mínimo


In [20]:
# Distribuição por setor
if 'setor' in df.columns and len(df) > 0:
    print("\n📊 Distribuição por SETOR:")
    sector_counts = df.group_by('setor').len().sort('len', descending=True)
    for row in sector_counts.iter_rows():
        setor, count = row
        pct = (count / total * 100) if total > 0 else 0
        print(f"   {setor}: {count} ({pct:.1f}%)")



📊 Distribuição por SETOR:
   Energia: 5 (16.1%)
   Consumo: 5 (16.1%)
   Financeiro: 5 (16.1%)
   Commodities: 5 (16.1%)
   Indústria: 4 (12.9%)
   Saúde: 3 (9.7%)
   Utilidades: 2 (6.5%)
   Tecnologia: 2 (6.5%)


In [21]:
# Distribuição por volatilidade
if 'volatility_class' in df.columns and len(df) > 0:
    print("\n📊 Distribuição por VOLATILIDADE:")
    for vol_class in ['BAIXA', 'MEDIA', 'ALTA']:
        count = df.filter(pl.col('volatility_class') == vol_class).height
        pct = (count / total * 100) if total > 0 else 0
        print(f"   {vol_class}: {count} ({pct:.1f}%)")



📊 Distribuição por VOLATILIDADE:
   BAIXA: 9 (29.0%)
   MEDIA: 16 (51.6%)
   ALTA: 6 (19.4%)


In [22]:
# Distribuição por liquidez
if 'liquidity_class' in df.columns and len(df) > 0:
    print("\n📊 Distribuição por LIQUIDEZ:")
    for liq_class in ['BAIXA', 'MEDIA', 'ALTA']:
        count = df.filter(pl.col('liquidity_class') == liq_class).height
        pct = (count / total * 100) if total > 0 else 0
        print(f"   {liq_class}: {count} ({pct:.1f}%)")



📊 Distribuição por LIQUIDEZ:
   BAIXA: 0 (0.0%)
   MEDIA: 10 (32.3%)
   ALTA: 21 (67.7%)


## 7. Metadados do Pipeline


In [23]:
# Carrega metadados
metadata = get_pipeline_metadata(output_path)

if metadata:
    print("\n📋 Metadados do Pipeline:")
    print(f"   Execução: {metadata.execution_date}")
    print(f"   Registros entrada: {metadata.input_record_count:,}")
    print(f"   Registros saída: {metadata.output_record_count:,}")
    print(f"   Filtros aplicados: {len(metadata.filters_applied)}")
    
    print("\n   📌 Filtros:")
    for f in metadata.filters_applied:
        print(f"      - {f}")
    
    if metadata.warnings:
        print(f"\n   ⚠️ Warnings:")
        for w in metadata.warnings:
            print(f"      - {w}")



📋 Metadados do Pipeline:
   Execução: 2025-12-02 17:07:50.907293
   Registros entrada: 63,626
   Registros saída: 31
   Filtros aplicados: 6

   📌 Filtros:
      - min_history_days >= 252
      - trading_days_ratio >= 0.9
      - avg_volume_21d >= R$ 5,000,000
      - avg_price >= R$ 5.00
      - tipo_instrumento in ['ACAO_ON', 'ACAO_PN', 'BDR']
      - max_names_per_sector < 6


## 8. Visualização dos Candidatos


In [24]:
# Exibe tabela completa
print(f"\n📋 LISTA COMPLETA DE CANDIDATOS ({len(df)} ativos):")
df



📋 LISTA COMPLETA DE CANDIDATOS (31 ativos):


ticker,tipo_instrumento,setor,date_first,date_last,history_days,trading_days_ratio_252d,avg_volume_21d_brl,avg_price_recent_brl,last_price,annualized_vol_60d,volatility_class,liquidity_class
str,str,str,date,date,u32,f64,f64,f64,f64,f64,str,str
"""CSAN3.SA""","""ACAO_ON""","""Energia""",2022-01-03,2025-12-01,980,1.0,4.6898e7,6.092381,6.15,0.567496,"""ALTA""","""ALTA"""
"""PETR4.SA""","""ACAO_PN""","""Commodities""",2022-01-03,2025-12-01,980,1.0,4.3878e7,31.980477,31.85,0.179207,"""BAIXA""","""ALTA"""
"""B3SA3.SA""","""ACAO_ON""","""Financeiro""",2022-01-03,2025-12-01,980,1.0,3.7703e7,13.842381,14.89,0.25815,"""MEDIA""","""ALTA"""
"""ABEV3.SA""","""ACAO_ON""","""Consumo""",2022-01-03,2025-12-01,980,1.0,3.0933e7,13.484286,14.01,0.195297,"""BAIXA""","""ALTA"""
"""ITSA4.SA""","""ACAO_PN""","""Financeiro""",2022-01-03,2025-12-01,980,1.0,3.0450e7,11.90381,12.24,0.161098,"""BAIXA""","""ALTA"""
…,…,…,…,…,…,…,…,…,…,…,…,…
"""MRVE3.SA""","""ACAO_ON""","""Indústria""",2022-01-03,2025-12-01,980,1.0,7.5379e6,8.3,9.27,0.431677,"""ALTA""","""MEDIA"""
"""RDOR3.SA""","""ACAO_ON""","""Saúde""",2022-01-03,2025-12-01,980,1.0,6.9668e6,45.51,45.700001,0.261548,"""MEDIA""","""MEDIA"""
"""TIMS3.SA""","""ACAO_ON""","""Tecnologia""",2022-01-03,2025-12-01,980,1.0,5.9648e6,24.728571,24.4,0.198139,"""BAIXA""","""MEDIA"""


---

## 🎯 Próximos Passos

### Se dentro do intervalo (60-80):
1. ✅ Revisar distribuição setorial
2. ✅ Revisar perfil de risco (volatilidade)
3. ➡️ Prosseguir para **TASK_011** (seleção dos 30 supervisionados)

### Se fora do intervalo:
1. Ajustar tickers em `config/experiments/universe_data_sources_v1.yaml`
2. Ajustar filtros em `config/experiments/universe_selection_rules_v1.yaml`
3. Re-executar este notebook

---

### Arquivos de Configuração:
- **Tickers:** `config/experiments/universe_data_sources_v1.yaml`
- **Filtros:** `config/experiments/universe_selection_rules_v1.yaml`

### Arquivos Gerados:
- **Pré-lista:** `data/universe/UNIVERSE_CANDIDATES.parquet`
- **CSV:** `data/universe/UNIVERSE_CANDIDATES.csv`
